## conversational Q&A Chatboat

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv    ("GROQ_PAI_KEY")

# cgatgroq
from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key=groq_api_key, model='Llama3-8b-8192')

In [2]:
# !pip install bs4

In [3]:
## assign huggingface api key
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# !pip install pysqlite3-binary

In [5]:
import sys
__import__('pysqlite3')
import pysqlite3
sys.modules['sqlite3'] = sys.modules["pysqlite3"]

from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
import bs4
loader = WebBaseLoader(
    web_path="https://lilianweng.github.io/posts/2023-06-23-agent/",
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", 'post-header')
        )
    )
)

docs = loader.load()


In [7]:
# !pip install pysqlite3-binary

In [8]:
import sys
__import__('pysqlite3')
import pysqlite3
sys.modules['sqlite3'] = sys.modules["pysqlite3"]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever()


In [9]:
## create a system prompt
system_prompt = (
    "You are a assistant for question answering task."
    "Use the following pieces of retrived context to answer."
    "the question. If you don't know the answer say that you"
    "don't know. use these sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

## create a chat prompt templete
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ("human", "{input}")
])



In [10]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [11]:
response = rag_chain.invoke({"input": "what is self-reflection?"})
response

{'input': 'what is self-reflection?',
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 3. Illustration of the Reflexion framework. (Image source: Shinn & Labash, 2023)\nThe heuristic function determines when the trajectory is inefficient or contains hallucination and should be stopped. Inefficient planning refers to trajectories that take too long without success. Hallucination is defined as encountering a sequence of consecutive identical actions that lead to the same observation in the environment.\nSelf-reflection is created by showing two-shot examples to LLM and each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan). Then reflections are added into the agent’s working memory, up to three, to be used as context for querying LLM.'),
  Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Memory stream: is a long-term memor

In [12]:
response['answer']

'According to the provided context, self-reflection is a vital aspect that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.'

In [13]:
rag_chain.invoke({"input": "how do i achieve it?"})

{'input': 'how do i achieve it?',
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Resources:\n1. Internet access for searches and information gathering.\n2. Long Term memory management.\n3. GPT-3.5 powered Agents for delegation of simple tasks.\n4. File output.\n\nPerformance Evaluation:\n1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.\n2. Constructively self-criticize your big-picture behavior constantly.\n3. Reflect on past decisions and strategies to refine your approach.\n4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.'),
  Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 11. Illustration of how HuggingGPT works. (Image source: Shen et al. 2023)\nThe system comprises of 4 stages:\n(1) Task planning: LLM works as the brain and parses the user req

## Adding Chat History

In [36]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
    ])


In [37]:
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7d3ebc3cfbf0>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChu

In [38]:
qa_prompt = ChatPromptTemplate([
    ("system", system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

In [39]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [40]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
question = "what is self-reflection?"

response1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response1['answer'])
]) 


In [41]:
response1['answer']

'According to the text, self-reflection is a mechanism that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It is a vital aspect that plays a crucial role in real-world tasks where trial and error are inevitable.'

In [42]:
question2 = "tell me more about it?"
response2 = rag_chain.invoke({"input":question2, "chat_history": chat_history})
print(response2['answer'])

According to the text, self-reflection is created by showing two-shot examples to LLM (Large Language Model) and each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan). Then reflections are added into the agent's working memory, up to three, to be used as context for querying LLM.


In [49]:
chat_history[1].content

'According to the text, self-reflection is a mechanism that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It is a vital aspect that plays a crucial role in real-world tasks where trial and error are inevitable.'

## create a session id to get same resposne for a time period

In [50]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

session_id_store = {}

def get_session_history(session_id: str)->BaseChatMessageHistory:
    if session_id not in session_id_store:
        session_id_store[session_id] = ChatMessageHistory()
    return session_id_store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key='input',
    history_messages_key='chat_history',
    output_messages_key='answer'
)

In [52]:
response = conversational_rag_chain.invoke(
    {"input": "What is task decompisition?"},
    config={
        "configurable": {"session_id":"chat1"}
            }
)

In [53]:
response['answer']

'According to the provided context, Task Decomposition is the process of breaking down a complicated task into smaller and simpler steps. This is achieved through techniques such as Chain of Thought (CoT) and Tree of Thoughts, which help to decompose hard tasks into manageable tasks, allowing the agent to plan ahead and think step by step.'

In [54]:
response = conversational_rag_chain.invoke(
    {"input": "What is self_reflection?"},
    config={
        "configurable": {"session_id":"chat1"}
            }
)

In [55]:
response['answer']

'According to the provided context, Self-Reflection is a mechanism that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It is a process that involves reflecting on past actions and their outcomes, and using this reflection to guide future changes in the plan.'

In [56]:
response2 = conversational_rag_chain.invoke(
    {"input": "tell me more about it?"},
    config={
        "configurable": {"session_id":"chat1"}
            }
)

In [57]:
response2['answer']

"According to the context, Self-Reflection is a process that involves showing the agent a pair of examples, consisting of a failed trajectory and an ideal reflection, to guide future changes in the plan. This process is used to improve the agent's decision-making by refining past actions and correcting previous mistakes.\n\nIn the Reflexion framework, self-reflection is created by showing two-shot examples to LLM (Large Language Model) and each example is a pair of (failed trajectory, ideal reflection) for guiding future changes in the plan. Then, reflections are added into the agent's working memory, up to three, to be used as context for querying LLM."

In [65]:
session_id_store['chat1']

InMemoryChatMessageHistory(messages=[HumanMessage(content='What is task decompisition?', additional_kwargs={}, response_metadata={}), AIMessage(content='According to the provided context, Task Decomposition is the process of breaking down a complicated task into smaller and simpler steps. This is achieved through techniques such as Chain of Thought (CoT) and Tree of Thoughts, which help to decompose hard tasks into manageable tasks, allowing the agent to plan ahead and think step by step.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is self_reflection?', additional_kwargs={}, response_metadata={}), AIMessage(content='According to the provided context, Self-Reflection is a mechanism that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It is a process that involves reflecting on past actions and their outcomes, and using this reflection to guide future changes in the plan.', additional_kwargs